In [44]:
import pandas as pd
import warnings
import hvplot.pandas
import warnings
import pickle
warnings.filterwarnings('ignore')

In [45]:
mapping_dict ={
'Mining': 'Materials',
 'Ownership of dwellings': 'Real Estate',
 'Financial and insurance services': 'Financials' ,
 'Construction': 'Industrials',
 'Health care and social assistance': 'Health Care' ,
 'Professional, scientific and technical services': 'Information Technology' ,
}

In [46]:
#read ASX data 

In [47]:
asx_df= pd.read_csv('./data/asx_all_info.csv')

In [48]:
asx_df["Code"] = asx_df['Code'].str.replace('ASX:', '', regex=True)#str.replace('[^\w\s]','')

In [49]:
asx_df['MktCap'] =asx_df['Market Cap'].replace({'K': '*1e3', 'M': '*1e6', 'B': '*1e9'}, regex=True).map(pd.eval).astype(float)

In [50]:
asx_df

,Code,Company,Market Cap,Sector,MktCap
0,CBA,Commonwealth Bank,180.83B,Financials,1.808300e+11
1,BHP,BHP Group Limited,143.91B,Materials,1.439100e+11
2,CSL,CSL,133.16B,Health Care,1.331600e+11
3,WBC,Westpac Banking Corp,97.69B,Financials,9.769000e+10
4,NAB,National Australia Bank,87.88B,Financials,8.788000e+10
...,...,...,...,...,...
272,OBL,Omni Bridgeway Limited,1.04B,Financials,1.040000e+09
273,DUI,Diversified United Investment Limited,1.02B,Financials,1.020000e+09
274,IDX,Integral Diagnostics Limited,1.02B,Health Care,1.020000e+09
275,OML,oOh!media Limited,1.01B,Communication Services,1.010000e+09


In [51]:
asx_top_50 = asx_df.sort_values(by="MktCap", ascending=False).head(50)

In [52]:
top_50_group = asx_top_50.groupby(["Sector"])

In [53]:
groups = top_50_group.groups

In [54]:
groups.keys()

dict_keys(['Communication Services', 'Consumer Discretionary', 'Consumer Staples', 'Energy', 'Financials', 'Health Care', 'Industrials', 'Information Technology', 'Materials', 'Real Estate', 'Utilities'])

In [55]:
top_50_group.get_group("Consumer Discretionary")

,Code,Company,Market Cap,Sector,MktCap
7,WES,Wesfarmers Limited,62.09B,Consumer Discretionary,6.209000e+10
15,ALL,Aristocrat Leisure,25.89B,Consumer Discretionary,2.589000e+10
42,TAH,Tabcorp Holdings,11.26B,Consumer Discretionary,1.126000e+10
47,DMP,Domino's Pizza,10.03B,Consumer Discretionary,1.003000e+10


## Load and analyse and prepare Australia statistics meta data

In [56]:
meta_data = pd.read_csv('./data/5204005_GVA_by_Industry_meta_mine.csv')

In [57]:
meta_data

,Data Item Description,Series Type,Series ID,Series Start,Series End,No. Obs.,Unit,Data Type,Freq.,Collection Month
0,"Agriculture, forestry and fishing (A) ; Agric...",Original,A3347243W,Jun-75,Jun-20,46,$ Millions,DERIVED,Annual,6
1,"Agriculture, forestry and fishing (A) ; Fores...",Original,A3347244X,Jun-75,Jun-20,46,$ Millions,DERIVED,Annual,6
2,"Agriculture, forestry and fishing (A) ; Chain...",Original,A3347242V,Jun-75,Jun-20,46,$ Millions,DERIVED,Annual,6
3,Mining (B) ; Coal mining: Chain volume measur...,Original,A3605348C,Jun-86,Jun-20,35,$ Millions,DERIVED,Annual,6
4,Mining (B) ; Oil and gas extraction: Chain vo...,Original,A3605349F,Jun-86,Jun-20,35,$ Millions,DERIVED,Annual,6
...,...,...,...,...,...,...,...,...,...,...
149,Health care and social assistance (Q) ; Perce...,Original,A3347212F,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
150,Arts and recreation services (R) ; Percentage...,Original,A3347213J,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
151,Other services (S) ; Percentage of GVA at bas...,Original,A3347214K,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
152,Ownership of dwellings ; Percentage of GVA at...,Original,A2421036A,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6


In [58]:
df_indexer = meta_data["Data Item Description"].str.split(pat=";", expand=True)
df_indexer= df_indexer.drop(columns= 2)
df_indexer =df_indexer.rename(columns={'0':"industry", '1': "indicator"})

In [59]:
index_m =pd.MultiIndex.from_frame(df_indexer)
meta_data.set_index(index_m,drop=True,inplace=True)

In [60]:
meta_data.drop(columns="Data Item Description",inplace=True)

In [61]:
meta_data

Series Type  \
0                                      1                                                                
Agriculture, forestry and fishing (A)    Agriculture: Chain volume measures                  Original   
                                         Forestry and fishing: Chain volume measures         Original   
                                         Chain volume measures                               Original   
Mining (B)                               Coal mining: Chain volume measures                  Original   
                                         Oil and gas extraction: Chain volume measures       Original   
...                                                                                               ...   
Health care and social assistance (Q)    Percentage of GVA at basic prices, current pr...    Original   
Arts and recreation services (R)         Percentage of GVA at basic prices, current pr...    Original   
Other services (S)                       Percentage of GVA at basic prices, current pr...    Original   
Ownership of dwellings                   Percentage of GVA at basic prices, current pr...    Original   
All industries                           Percentage of GVA at basic prices, current pr...    Original   

                                                                                           Series ID  \
0                                      1                                                               
Agriculture, forestry and fishing (A)    Agriculture: Chain volume measures                A3347243W   
                                         Forestry and fishing: Chain volume measures       A3347244X   
                                         Chain volume measures                             A3347242V   
Mining (B)                               Coal mining: Chain volume measures                A3605348C   
                                         Oil and gas extraction: Chain volume measures     A3605349F   
...                                                                                              ...   
Health care and social assistance (Q)    Percentage of GVA at basic prices, current pr...  A3347212F   
Arts and recreation services (R)         Percentage of GVA at basic prices, current pr...  A3347213J   
Other services (S)                       Percentage of GVA at basic prices, current pr...  A3347214K   
Ownership of dwellings                   Percentage of GVA at basic prices, current pr...  A2421036A   
All industries                           Percentage of GVA at basic prices, current pr...  A2421037C   

                                                                                          Series Start  \
0                                      1                                                                 
Agriculture, forestry and fishing (A)    Agriculture: Chain volume measures                     Jun-75   
                                         Forestry and fishing: Chain volume measures            Jun-75   
                                         Chain volume measures                                  Jun-75   
Mining (B)                               Coal mining: Chain volume measures                     Jun-86   
                                         Oil and gas extraction: Chain volume measures          Jun-86   
...                                                                                                ...   
Health care and social assistance (Q)    Percentage of GVA at basic prices, current pr...       Jun-90   
Arts and recreation services (R)         Percentage of GVA at basic prices, current pr...       Jun-90   
Other services (S)                       Percentage of GVA at basic prices, current pr...       Jun-90   
Ownership of dwellings                   Percentage of GVA at basic prices, current pr...       Jun-90   
All industries                           Percentage of GVA at basic prices, current pr...       Jun

In [62]:
 analysis_df = meta_data[meta_data['Unit'] == 'proportion']

In [63]:
analysis_df

,,Series Type,Series ID,Series Start,Series End,No. Obs.,Unit,Data Type,Freq.,Collection Month
0,1,,,,,,,,,
"Agriculture, forestry and fishing (A)","Percentage of GVA at basic prices, current prices",Original,A3347196T,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
Mining (B),"Percentage of GVA at basic prices, current prices",Original,A3347197V,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
Manufacturing (C),"Percentage of GVA at basic prices, current prices",Original,A3347198W,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
"Electricity, gas, water and waste services (D)","Percentage of GVA at basic prices, current prices",Original,A3347199X,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
Construction (E),"Percentage of GVA at basic prices, current prices",Original,A3347200W,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
Wholesale trade (F),"Percentage of GVA at basic prices, current prices",Original,A3347201X,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
Retail trade (G),"Percentage of GVA at basic prices, current prices",Original,A3347202A,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
Accommodation and food services (H),"Percentage of GVA at basic prices, current prices",Original,A3347203C,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6
"Transport, postal and warehousing (I)","Percentage of GVA at basic prices, current prices",Original,A3347204F,Jun-90,Jun-20,31,proportion,DERIVED,Annual,6


## Load and analyse indicators

In [64]:
ind_data = pd.read_csv('./data/5204005_GVA_by_Industry_data_mine.csv',index_col='Date',parse_dates=True)

In [65]:
contri_df = ind_data[ind_data.columns.intersection(analysis_df['Series ID'].to_list())]

In [66]:
for col in contri_df.columns:
    name = analysis_df[analysis_df['Series ID'] == col].index[0][0]
    contri_df.rename(columns={col:name},inplace=True)

In [67]:
#change to shape of index as periods
contri_df.set_index(pd.PeriodIndex(pd.date_range("2000-Jun", periods=21, freq="A-JUN"), freq="y"), drop=True,inplace=True)

In [68]:
contri_df.drop(columns=contri_df.columns[-1]).hvplot.bar(stacked=True, height=800, width = 1800, legend='right',title="Contribution by Industry to GDP")

:Bars   [index,Variable]   (value)

In [69]:
contri_df.loc["2018":"2020"].T.sort_values(by="2020",ascending=False)

,2018,2019,2020
All industries,100.0,100.0,100.0
Mining (B),8.8,10.6,11.1
Ownership of dwellings,8.9,8.7,8.9
Financial and insurance services (K),9.2,9.0,8.9
Construction (E),8.2,7.9,7.7
Health care and social assistance (Q),7.4,7.6,7.7
"Professional, scientific and technical services (M)",7.3,7.3,7.6
Manufacturing (C),6.2,6.0,6.1
Public administration and safety (O),5.5,5.6,5.7
Education and training (P),5.1,5.0,5.1


In [70]:
#select top industries

In [71]:
short_industries = contri_df.loc["2018":"2020"].T.sort_values(by="2020",ascending=False).head(7)

In [72]:
short_industries.drop("All industries ",inplace= True)

In [73]:
short_industries

,2018,2019,2020
Mining (B),8.8,10.6,11.1
Ownership of dwellings,8.9,8.7,8.9
Financial and insurance services (K),9.2,9.0,8.9
Construction (E),8.2,7.9,7.7
Health care and social assistance (Q),7.4,7.6,7.7
"Professional, scientific and technical services (M)",7.3,7.3,7.6


In [74]:
short_industries['ASX Mapping'] = short_industries.index.str.replace(r"\(.*\)","").str.strip().map(mapping_dict);

In [75]:
short_industries

,2018,2019,2020,ASX Mapping
Mining (B),8.8,10.6,11.1,Materials
Ownership of dwellings,8.9,8.7,8.9,Real Estate
Financial and insurance services (K),9.2,9.0,8.9,Financials
Construction (E),8.2,7.9,7.7,Industrials
Health care and social assistance (Q),7.4,7.6,7.7,Health Care
"Professional, scientific and technical services (M)",7.3,7.3,7.6,Information Technology


In [76]:
top_50_group.get_group("Materials")

,Code,Company,Market Cap,Sector,MktCap
1,BHP,BHP Group Limited,143.91B,Materials,1.439100e+11
6,FMG,Fortescue Metals Group,69.74B,Materials,6.974000e+10
10,RIO,Rio Tinto,46.52B,Materials,4.652000e+10
18,NCM,Newcrest Mining Limited,22.56B,Materials,2.256000e+10
21,JHX,James Hardie,19.15B,Materials,1.915000e+10
34,S32,South32,14.11B,Materials,1.411000e+10
35,AMC,Amcor,13.08B,Materials,1.308000e+10
37,NST,Northern Star Resources,12.72B,Materials,1.272000e+10
40,BSL,BlueScope Steel Limited,11.32B,Materials,1.132000e+10


In [77]:
top_50_group.get_group("Industrials")["Code"].apply(lambda x: "{}{}".format(x,'.AX'))

12    TCL.AX
24    SYD.AX
26    BXB.AX
29    REH.AX
46    AIA.AX
Name: Code, dtype: object

In [78]:
industry_stocks={}
for ind in short_industries['ASX Mapping']:
    industry_stocks[ind] = top_50_group.get_group(ind)["Code"].apply(lambda x: "{}{}".format(x,'.AX')).to_list()

In [79]:
industry_stocks

{'Materials': ['BHP.AX',
  'FMG.AX',
  'RIO.AX',
  'NCM.AX',
  'JHX.AX',
  'S32.AX',
  'AMC.AX',
  'NST.AX',
  'BSL.AX'],
 'Real Estate': ['GMG.AX', 'SCG.AX', 'MGR.AX', 'DXS.AX', 'SGP.AX'],
 'Financials': ['CBA.AX',
  'WBC.AX',
  'NAB.AX',
  'ANZ.AX',
  'MQG.AX',
  'QBE.AX',
  'ASX.AX',
  'SUN.AX',
  'IAG.AX'],
 'Industrials': ['TCL.AX', 'SYD.AX', 'BXB.AX', 'REH.AX', 'AIA.AX'],
 'Health Care': ['CSL.AX', 'SHL.AX', 'FPH.AX', 'COH.AX', 'RHC.AX', 'RMD.AX'],
 'Information Technology': ['APT.AX', 'XRO.AX', 'CPU.AX']}

In [80]:
# Store data (serialize)
with open('./data/pickles/industry_stocks.pickle', 'wb') as handle:
    pickle.dump(industry_stocks, handle, protocol=pickle.HIGHEST_PROTOCOL)